In [19]:
import numpy as np

# S-Box for substitution
S_BOX = {
    '0': 'a', '1': '4', '2': '3', '3': 'b',
    '4': '8', '5': 'e', '6': '2', '7': 'c',
    '8': '5', '9': '7', 'a': '6', 'b': 'f',
    'c': '0', 'd': '1', 'e': '9', 'f': 'd'
}
INV_S_BOX = {v: k for k, v in S_BOX.items()}

# MixColumns matrix
MIX_COLUMNS_MATRIX = np.array([
    [1, 0, 1, 0, 0, 0, 1, 1],
    [1, 1, 0, 1, 0, 0, 0, 1],
    [1, 1, 1, 0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0, 1, 1, 1],
    [0, 0, 1, 1, 1, 0, 1, 0],
    [0, 0, 0, 1, 1, 1, 0, 1],
    [1, 0, 0, 0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0, 1, 0, 1]
])

def reverse(column):
    """Interchanges the two entries in the column."""
    return column[::-1]


def generate_round_keys(initial_key, rounds=4):
    
    
    w = [initial_key[:, 0], initial_key[:, 1]]  # Initialize w0 and w1
    
    for i in range(1, rounds + 1):
        y_i = np.array([(1 << (i - 1)), 0], dtype=np.uint8)  # Compute yi
        w_2i = w[2 * (i - 1)] ^ sub_nibbles(reverse(w[2 * (i - 1) + 1])) ^ y_i
        w_2i_1 = w[2 * (i - 1) + 1] ^ w_2i
        
        w_2i=[int(hex(num), 16) for num in w_2i]
        w_2i_1=[int(hex(num), 16) for num in w_2i_1]
        w.append(w_2i)
        w.append(w_2i_1)
    
    round_keys = [np.column_stack((w[2 * i], w[2 * i + 1])) for i in range(1,rounds+1)]
    return round_keys


def sub_nibbles(state):
    """Applies the S-box transformation to a 2x2 matrix."""
    return np.vectorize(lambda n: int(S_BOX[hex(n)[2:]], 16))(state)

def inv_sub_nibbles(state):
    """Applies the inverse S-box transformation to a 2x2 matrix."""
    return np.vectorize(lambda n: int(INV_S_BOX[hex(n)[2:]], 16))(state)

def mix_columns(state,state_bin):
    
    
    mixed = np.dot(MIX_COLUMNS_MATRIX, state_bin) % 2
    
    return mixed

def hex_array_to_binary_array(hex_list):
    return [[bit for bit in bin(int(hex_val, 16))[2:].zfill(len(hex_val) * 4)] for hex_val in hex_list]

def binary_int_list_to_hex_int_list(binary_list):


    bList=[]
    for i in range(len(binary_list)):
        bList.append(int("".join(map(str, list(binary_list[i]))), 2))
    

    return [int(hex(num), 16) for num in bList]




def add_round_key(state, key):
    return np.bitwise_xor(state,key)

def shift_rows(state):
    state=list(state)
    return np.array([[state[0][0], state[0][1]], [state[1][1], state[1][0]]])

def encrypt_block(plaintext, key):


    state = plaintext[:]

    state= np.array([[state[0], state[2]], [state[1], state[3]]], dtype=np.uint8)

    key = np.array([[key[0], key[2]], [key[1], key[3]]], dtype=np.uint8)
    
    round_key=key

    state_1d = np.array([state[0, 0], state[1 ,0], state[0, 1], state[1, 1]])
    print(f"Initial Plaintext: {[hex(x)[2:].zfill(2) for x in state_1d]}")


    round_keys=generate_round_keys(key)

    print(round_keys)

    print(state)
    state = add_round_key(state, round_key)

    state_1d = np.array([state[0, 0], state[1 ,0], state[0, 1], state[1, 1]])
    print(f"After First AddRoundKey: {[hex(x)[2:].zfill(2) for x in state_1d]}")
    for i  in range(4):
  
        state = sub_nibbles(state)
        state_1d = np.array([state[0, 0], state[1 ,0], state[0, 1], state[1, 1]])
        print(f"Round {i+1} - After SubNibbles: {[hex(x)[2:].zfill(2) for x in state_1d]}")
        state = shift_rows(state)
        state_1d = np.array([state[0, 0], state[1 ,0], state[0, 1], state[1, 1]])
        print(f"Round {i+1} - After ShiftRows: {[hex(x)[2:].zfill(2) for x in state_1d]}")
        if i < 3:  # Skip MixColumns in the last round

            #8*2 matrix
            
            binary_state=hex_array_to_binary_array([hex(x)[2:].zfill(1) for x in state_1d])
            print(binary_state)
            


            zero_array = np.zeros((8, 2), dtype=int)
            m=0
            for j in range(2):
                for k in range(8):
                        
                        zero_array[k][j]=binary_state[int(k/4)+(j)%2+m][k%4]
                m=1
               
            print(zero_array)



            tempstate = mix_columns(state,zero_array)


            stateT=[]

            stateT.append(tempstate[:4, 0])
            stateT.append(tempstate[4:8, 0])
            stateT.append(tempstate[:4, 1])
            stateT.append(tempstate[4:8, 1])


            stateT=binary_int_list_to_hex_int_list(stateT)

            state= np.array([[stateT[0], stateT[2]], [stateT[1], stateT[3]]], dtype=np.uint8)

            state_1d = np.array([state[0, 0], state[1 ,0], state[0, 1], state[1, 1]])
            print(f"Round {i+1} - After MixColumns: {[hex(x)[2:].zfill(2) for x in state_1d]}")


        state = add_round_key(state, round_keys[i])

        state_1d = np.array([state[0, 0], state[1 ,0], state[0, 1], state[1, 1]])
        print(f"Round {i+1} - After AddRoundKey: {[hex(x)[2:].zfill(2) for x in state_1d]}")

    state_1d = np.array([state[0, 0], state[1 ,0], state[0, 1], state[1, 1]])
    print(f"Final Ciphertext: {[hex(x)[2:].zfill(2) for x in state_1d]}")
    return np.array([state[0, 0], state[1 ,0], state[0, 1], state[1, 1]])

def bit_diff_count(a, b):
    return sum(bin(x ^ y).count('1') for x, y in zip(a, b))

def bit_diff_count(a, b):
    return sum(bin(x ^ y).count('1') for x, y in zip(a, b))

def strict_avalanche_effect(plaintext, key):
    original_ciphertext = encrypt_block(plaintext, key)
    key_size = len(key) * 4  # Assuming each key byte is 8 bits
    total_bits = len(original_ciphertext) * 4
    
    total_diff = sum(
        bit_diff_count(encrypt_block(plaintext, [(k if j // 4 != i else k ^ (1 << (j % 4))) for i, k in enumerate(key)]), original_ciphertext)
        for j in range(key_size)
    )
    
    return (total_diff / (key_size * total_bits)) * 100


def completeness_check(key, plaintext):
    key_bits = ''.join(f'{k:04b}' for k in key)  # Convert key to binary string
    plaintext_bits = ''.join(f'{p:04b}' for p in plaintext)  # Convert plaintext to binary string

    key_length = len(key_bits)
    plaintext_length = len(plaintext_bits)

    influence_matrix = [[0] * plaintext_length for _ in range(key_length)]

    for i in range(key_length):
        modified_key_bits = list(key_bits)
        modified_key_bits[i] = '1' if key_bits[i] == '0' else '0'  # Flip one bit in the key

        modified_key = [int("".join(modified_key_bits[j:j+4]), 2) for j in range(0, key_length, 4)]

        modified_plaintext = encrypt_block(plaintext,modified_key)

        modified_plaintext_bits = ''.join(f'{p:04b}' for p in modified_plaintext)

        for j in range(plaintext_length):
            if modified_plaintext_bits[j] != plaintext_bits[j]:
                influence_matrix[i][j] = 1  # Mark influence

    return influence_matrix
def evaluate_security():
    key = [0x06, 0x0B, 0x05, 0x0D]
    plaintext = [0x2, 0xC, 0xA, 0x5]
    ciphertext = encrypt_block(plaintext, key)
    altered_plaintext = [0x2, 0xC, 0xA, 0x6]  # Change one bit
    altered_ciphertext = encrypt_block(altered_plaintext, key)
    
    completeness = completeness_check(key, plaintext)


    avalanche = bit_diff_count(ciphertext, altered_ciphertext) / (len(ciphertext) * 4) * 100
    
    integrity = bit_diff_count(plaintext, ciphertext) / (len(ciphertext) * 4) * 100


    strict_avalanche = strict_avalanche_effect(plaintext, key)
    
    for row in completeness:
        print(row)
    print(f"Avalanche Effect: {avalanche:.2f}% bits changed when input modified")
    print(f"Integrity: {integrity:.2f}% bit difference between plaintext and ciphertext")
    print(f"Strict Avalanche Effect: {strict_avalanche:.2f}% bits changed when key bit flipped")

evaluate_security()

Initial Plaintext: ['02', '0c', '0a', '05']
[array([[6, 3],
       [5, 8]]), array([[ 1,  2],
       [14,  6]]), array([[ 7,  5],
       [13, 11]]), array([[0, 5],
       [3, 8]])]
[[ 2 10]
 [12  5]]
After First AddRoundKey: ['04', '07', '0f', '08']
Round 1 - After SubNibbles: ['08', '0c', '0d', '05']
Round 1 - After ShiftRows: ['08', '05', '0d', '0c']
[['1', '0', '0', '0'], ['0', '1', '0', '1'], ['1', '1', '0', '1'], ['1', '1', '0', '0']]
[[1 1]
 [0 1]
 [0 0]
 [0 1]
 [0 1]
 [1 1]
 [0 0]
 [1 0]]
Round 1 - After MixColumns: ['02', '00', '0f', '07']
Round 1 - After AddRoundKey: ['04', '05', '0c', '0f']
Round 2 - After SubNibbles: ['08', '0e', '00', '0d']
Round 2 - After ShiftRows: ['08', '0d', '00', '0e']
[['1', '0', '0', '0'], ['1', '1', '0', '1'], ['0', '0', '0', '0'], ['1', '1', '1', '0']]
[[1 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [0 1]
 [1 0]]
Round 2 - After MixColumns: ['00', '0e', '0a', '03']
Round 2 - After AddRoundKey: ['01', '00', '08', '05']
Round 3 - After SubNibbles: ['04',

In [20]:
import random
import numpy as np



def generate_random_text(length=16):
    binary_string= format(random.getrandbits(length), f'0{length}b')
    hex_string = format(int(binary_string, 16), '04X')  # Convert to 4-digit hex string
    return list(hex_string)

def xor_strings(s1, s2):
    return ''.join(str(int(a) ^ int(b)) for a, b in zip(s1, s2))

def generate_random_binary(length=16):
    return format(random.getrandbits(length), f'0{length}b')


def flip_one_bit(hex_list):
    # Convert hex list to a binary string
    binary_string = ''.join(format(int(h, 16), '04b') for h in hex_list)
    
    # Choose a random bit to flip
    bit_index = random.randint(0, len(binary_string) - 1)
    
    # Flip the bit
    flipped_binary = (
        binary_string[:bit_index] +
        ('1' if binary_string[bit_index] == '0' else '0') +
        binary_string[bit_index+1:]
    )
    
    # Convert back to hex list
    new_hex_list = [format(int(flipped_binary[i:i+4], 2), 'X') for i in range(0, len(flipped_binary), 4)]
    
    return new_hex_list







def generate_avalanche_plaintext(samples=100):
    dataset = []
    for _ in range(samples):
        plaintext = generate_random_text()
        flipped_plaintext = flip_one_bit(plaintext)
        key = generate_random_text()
        ciphertext1 = encrypt_block(plaintext, key)
        ciphertext2 = encrypt_block(flipped_plaintext, key)
        dataset.append(xor_strings(ciphertext1, ciphertext2))
    return dataset

def generate_avalanche_key(samples=100):
    dataset = []
    for _ in range(samples):
        key = generate_random_text()
        flipped_key = flip_one_bit(key)
        plaintext = generate_random_text()
        ciphertext1 = encrypt_block(plaintext, key)
        ciphertext2 = encrypt_block(plaintext, flipped_key)
        dataset.append(xor_strings(ciphertext1, ciphertext2))
    return dataset

def generate_plaintext_ciphertext_correlation(samples=100):
    dataset = []
    key = generate_random_text()
    for _ in range(samples):
        plaintext = generate_random_text()
        ciphertext = encrypt_block(plaintext, key)
        dataset.append(xor_strings(plaintext, ciphertext))
    return dataset

def generate_random_dataset(samples=100, length=16):
    return [generate_random_binary(length) for _ in range(samples)]

def generate_cbc_mode_dataset(samples=100):
    dataset = []
    for _ in range(samples):
        key = generate_random_text()
        iv = generate_random_text()
        plaintext = generate_random_text()
        cipher = encrypt_block(plaintext, key)
        dataset.append(''.join(format(byte, '04b') for byte in cipher))
    return dataset

def generate_low_density_plaintext(samples=100, length=16, ones_ratio=0.1):
    dataset = []
    for _ in range(samples):
        num_ones = max(1, int(length * ones_ratio))
        binary = ['0'] * length
        ones_positions = random.sample(range(length), num_ones)
        for pos in ones_positions:
            binary[pos] = '1'
        dataset.append(''.join(binary))
    return dataset




def generate_high_density_plaintext(samples=100, length=16, zeros_ratio=0.1):
    dataset = []
    for _ in range(samples):
        num_zeros = max(1, int(length * zeros_ratio))
        binary = ['1'] * length
        zeros_positions = random.sample(range(length), num_zeros)
        for pos in zeros_positions:
            binary[pos] = '0'
        dataset.append(''.join(binary))
    return dataset

def generate_low_density_key(samples=100, length=16, ones_ratio=0.1):
      return generate_low_density_plaintext(samples, length, ones_ratio)
  
def generate_high_density_key(samples=100, length=16, zeros_ratio=0.1):
      return generate_high_density_plaintext(samples, length, zeros_ratio)

datasets = {
    "Avalanche Plaintext": generate_avalanche_plaintext(),
    "Avalanche Key": generate_avalanche_key(),
    "Plaintext-Ciphertext Correlation": generate_plaintext_ciphertext_correlation(),
    "Cipher Block Chaining (CBC) Mode": generate_cbc_mode_dataset(),
    "Random": generate_random_dataset(),
    "Low-Density Plaintext": generate_low_density_plaintext(),
    "High-Density Plaintext": generate_high_density_plaintext(),
    "Low-Density Key": generate_low_density_key(),
    "High-Density Key": generate_high_density_key()
}

# Print sample data
for name, data in datasets.items():
    print(f"\n{name} (Sample):")
    print(data[:5])



Initial Plaintext: ['01', '01', '00', '00']
[array([[4, 5],
       [4, 5]]), array([[ 8, 13],
       [10, 15]]), array([[ 1, 12],
       [11,  4]]), array([[ 1, 13],
       [11, 15]])]
[[1 0]
 [1 0]]
After First AddRoundKey: ['00', '01', '01', '01']
Round 1 - After SubNibbles: ['0a', '04', '04', '04']
Round 1 - After ShiftRows: ['0a', '04', '04', '04']
[['1', '0', '1', '0'], ['0', '1', '0', '0'], ['0', '1', '0', '0'], ['0', '1', '0', '0']]
[[1 0]
 [0 1]
 [1 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]]
Round 1 - After MixColumns: ['05', '0c', '06', '06']
Round 1 - After AddRoundKey: ['01', '08', '03', '03']
Round 2 - After SubNibbles: ['04', '05', '0b', '0b']
Round 2 - After ShiftRows: ['04', '0b', '0b', '05']
[['0', '1', '0', '0'], ['1', '0', '1', '1'], ['1', '0', '1', '1'], ['0', '1', '0', '1']]
[[0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 1]]
Round 2 - After MixColumns: ['01', '00', '0d', '04']
Round 2 - After AddRoundKey: ['09', '0a', '00', '0b']
Round 3 - After SubNibbles: ['07',

In [21]:
import math
from collections import Counter

def entropy_estimation(binary_sequence):
    """
    Estimates the entropy of a binary sequence using Shannon entropy.
    :param binary_sequence: A string of '0's and '1's
    :return: Estimated entropy value
    """
    length = len(binary_sequence)
    if length == 0:
        return 0
    
    # Count occurrences of '0' and '1'
    counts = Counter(binary_sequence)
    probabilities = [count / length for count in counts.values()]
    
    # Shannon entropy formula
    entropy = -sum(p * math.log2(p) for p in probabilities if p > 0)
    
    return entropy

datasets = {
    "Avalanche Plaintext": generate_avalanche_plaintext(),
    "Avalanche Key": generate_avalanche_key(),
    "Plaintext-Ciphertext Correlation": generate_plaintext_ciphertext_correlation(),
    "Cipher Block Chaining (CBC) Mode": generate_cbc_mode_dataset(),
    "Random": generate_random_dataset(),
    "Low-Density Plaintext": generate_low_density_plaintext(),
    "High-Density Plaintext": generate_high_density_plaintext(),
    "Low-Density Key": generate_low_density_key(),
    "High-Density Key": generate_high_density_key()
}


# Print sample data with entropy
def evaluate_entropy(datasets):
    for name, data in datasets.items():
        entropy_values = [entropy_estimation(seq) for seq in data]
        avg_entropy = sum(entropy_values) / len(entropy_values)
        print(f"\n{name} (Sample):")
        print(data[:5])
        print(f"Average Entropy: {avg_entropy:.4f}")

evaluate_entropy(datasets)


Initial Plaintext: ['01', '00', '00', '01']
[array([[10, 10],
       [10, 10]]), array([[14,  4],
       [12,  6]]), array([[ 8, 12],
       [ 4,  2]]), array([[ 3, 15],
       [ 4,  6]])]
[[1 0]
 [0 1]]
After First AddRoundKey: ['00', '00', '00', '01']
Round 1 - After SubNibbles: ['0a', '0a', '0a', '04']
Round 1 - After ShiftRows: ['0a', '04', '0a', '0a']
[['1', '0', '1', '0'], ['0', '1', '0', '0'], ['1', '0', '1', '0'], ['1', '0', '1', '0']]
[[1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 1]
 [1 0]
 [0 1]
 [0 0]]
Round 1 - After MixColumns: ['05', '0c', '0f', '0f']
Round 1 - After AddRoundKey: ['0f', '06', '05', '05']
Round 2 - After SubNibbles: ['0d', '02', '0e', '0e']
Round 2 - After ShiftRows: ['0d', '0e', '0e', '02']
[['1', '1', '0', '1'], ['1', '1', '1', '0'], ['1', '1', '1', '0'], ['0', '0', '1', '0']]
[[1 1]
 [1 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [0 0]]
Round 2 - After MixColumns: ['06', '0d', '0a', '00']
Round 2 - After AddRoundKey: ['08', '01', '0e', '06']
Round 3 - After SubNibbles: ['